In [1]:
import torch
import torch.nn as nn
import string

# 하이퍼파라미터 설정
# string.pritable을 이용하여 출력가능한 아스키 문자를 담은 문자열 상수
input_size = len(string.printable) 
hidden_size = 128
output_size = len(string.printable)
num_layers = 2
learning_rate = 0.01
num_epochs = 15

# 훈련 데이터 정의
input_str = 'ehlloowlrd'
output_str = 'hellowolrd'

all_chars = string.printable

# 문자를 인덱스로 변환
char2index = dict(zip(all_chars, range(len(all_chars))))
# 인덱스를 문자로 변환
index2char = dict(zip(range(len(all_chars)), all_chars)) 

# RNN 모델 정의
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        # 배치차원이 맨 앞으로 오도록 함
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) 
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        # RNN 순전파 수행
        out, hidden = self.rnn(x, hidden)
        # out의 크기를 (batch_size * sequence_length, hidden_size)로 변경
        out = out.reshape(out.size(0)*out.size(1), self.hidden_size)
        # out을 딥러닝 모델에 통과시켜 출력값 계산
        out = self.fc(out)
        return out, hidden

# 모델 인스턴스 생성
model = RNN(input_size, hidden_size, output_size, num_layers)

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 훈련 루프
for epoch in range(num_epochs):
    # 각 시점에서 hidden state가 유지되어야 하기 때문에 초기화를 위해 None 할당
    hidden = None
    
    # 입력과 출력 데이터 생성
    input_tensor = torch.tensor([[char2index[c] for c in input_str]])
    output_tensor = torch.tensor([char2index[c] for c in output_str])
    
    # 모델 입력을 위한 one-hot encoding
    input_one_hot = nn.functional.one_hot(input_tensor, num_classes=len(all_chars)).float()
    
    # 순전파와 역전파
    optimizer.zero_grad()
    output, hidden = model(input_one_hot, hidden)
    loss = criterion(output, output_tensor)
    loss.backward()
    optimizer.step()
    
    # 예측값 계산
    # 모델이 예측한 값 중 확률이 가장 높은 문자의 인덱스를 가져옴
    output_indices = output.argmax(dim=1)
    # output_indices의 값들을 하나씩 가져와 다시 문자형태로 변환 후 공백없이 문자열로 합쳐준다
    predicted_chars = ''.join([index2char[i.item()] for i in output_indices])
    
    # 정확도 계산
    accuracy = torch.mean((output_indices == output_tensor).float())
    
    # loss와 accuracy 출력
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%, Predicted String: {predicted_chars}')


Epoch [1/15], Loss: 4.6231, Accuracy: 0.00%, Predicted String: /i/i0y//i/
Epoch [2/15], Loss: 3.7210, Accuracy: 40.00%, Predicted String: llllloolll
Epoch [3/15], Loss: 2.2563, Accuracy: 40.00%, Predicted String: lllloooooo
Epoch [4/15], Loss: 1.8349, Accuracy: 30.00%, Predicted String: llllllllll
Epoch [5/15], Loss: 1.6239, Accuracy: 40.00%, Predicted String: llllloolll
Epoch [6/15], Loss: 1.4432, Accuracy: 40.00%, Predicted String: hlllllllll
Epoch [7/15], Loss: 1.3538, Accuracy: 50.00%, Predicted String: helooooooo
Epoch [8/15], Loss: 1.1346, Accuracy: 60.00%, Predicted String: helloooooo
Epoch [9/15], Loss: 1.0355, Accuracy: 50.00%, Predicted String: helllorlll
Epoch [10/15], Loss: 0.9447, Accuracy: 50.00%, Predicted String: helllollll
Epoch [11/15], Loss: 0.8768, Accuracy: 80.00%, Predicted String: hellowolow
Epoch [12/15], Loss: 0.8003, Accuracy: 80.00%, Predicted String: hellowoloo
Epoch [13/15], Loss: 0.6656, Accuracy: 70.00%, Predicted String: hellooollo
Epoch [14/15], Loss: 0

In [2]:
import torch
import torch.nn as nn
import string
import random

# 하이퍼파라미터 설정
# string.pritable을 이용하여 출력가능한 아스키 문자를 담은 문자열 상수
input_size = len(string.printable) 
hidden_size = 128
output_size = len(string.printable)
num_layers = 4
learning_rate = 0.01
num_epochs = 20

# 훈련 데이터 정의
input_str = 'hellowolrd'
output_str = 'hellowolrd'

# input_str과 output_str을 리스트로 변환
input_list = list(input_str)

# 리스트를 랜덤하게 섞음
random.shuffle(input_list)

# 섞인 리스트를 다시 문자열로 변환
shuffled_input_str = ''.join(input_list)

print('입력데이터 : ', shuffled_input_str)  # 예시 출력: "dheoowlllr"
print('출력데이터 : ', output_str)

all_chars = string.printable

# 문자를 인덱스로 변환
char2index = dict(zip(all_chars, range(len(all_chars))))
# 인덱스를 문자로 변환
index2char = dict(zip(range(len(all_chars)), all_chars)) 

# RNN 모델 정의
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        # 배치차원이 맨 앞으로 오도록 함
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) 
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        # RNN 순전파 수행
        out, hidden = self.rnn(x, hidden)
        # out의 크기를 (batch_size * sequence_length, hidden_size)로 변경
        out = out.reshape(out.size(0)*out.size(1), self.hidden_size)
        # out을 딥러닝 모델에 통과시켜 출력값 계산
        out = self.fc(out)
        return out, hidden

# 모델 인스턴스 생성
model = RNN(input_size, hidden_size, output_size, num_layers)

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 훈련 루프
for epoch in range(num_epochs):
    # 각 시점에서 hidden state가 유지되어야 하기 때문에 초기화를 위해 None 할당
    hidden = None
    
    # 입력과 출력 데이터 생성
    input_tensor = torch.tensor([[char2index[c] for c in input_str]])
    output_tensor = torch.tensor([char2index[c] for c in output_str])
    
    # 모델 입력을 위한 one-hot encoding
    input_one_hot = nn.functional.one_hot(input_tensor, num_classes=len(all_chars)).float()
    
    # 순전파와 역전파
    optimizer.zero_grad()
    output, hidden = model(input_one_hot, hidden)
    loss = criterion(output, output_tensor)
    loss.backward()
    optimizer.step()
    
    # 예측값 계산
    # 모델이 예측한 값 중 확률이 가장 높은 문자의 인덱스를 가져옴
    output_indices = output.argmax(dim=1)
    # output_indices의 값들을 하나씩 가져와 다시 문자형태로 변환 후 공백없이 문자열로 합쳐준다
    predicted_chars = ''.join([index2char[i.item()] for i in output_indices])
    
    # 정확도 계산
    accuracy = torch.mean((output_indices == output_tensor).float())
    
    # loss와 accuracy 출력
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%, Predicted String: {predicted_chars}')


입력데이터 :  elworoldlh
출력데이터 :  hellowolrd
Epoch [1/20], Loss: 4.5982, Accuracy: 0.00%, Predicted String: .TBTTBBBBT
Epoch [2/20], Loss: 3.4539, Accuracy: 50.00%, Predicted String: olllololll
Epoch [3/20], Loss: 2.1375, Accuracy: 30.00%, Predicted String: olllllllll
Epoch [4/20], Loss: 1.7844, Accuracy: 30.00%, Predicted String: llllllllll
Epoch [5/20], Loss: 1.5765, Accuracy: 40.00%, Predicted String: hllooooooo
Epoch [6/20], Loss: 1.3115, Accuracy: 60.00%, Predicted String: hellloolol
Epoch [7/20], Loss: 4.0802, Accuracy: 50.00%, Predicted String: hellloo4$v
Epoch [8/20], Loss: 1.6255, Accuracy: 40.00%, Predicted String: heellllloo
Epoch [9/20], Loss: 1.6184, Accuracy: 40.00%, Predicted String: heelllllll
Epoch [10/20], Loss: 1.3175, Accuracy: 60.00%, Predicted String: helloooooo
Epoch [11/20], Loss: 1.2191, Accuracy: 50.00%, Predicted String: helooooooo
Epoch [12/20], Loss: 1.0890, Accuracy: 60.00%, Predicted String: helloooooo
Epoch [13/20], Loss: 1.1168, Accuracy: 50.00%, Predicted S

In [3]:
import torch
import torch.nn as nn
import string
import random

# 하이퍼파라미터 설정
input_size = len(string.printable) 
hidden_size = 128
output_size = len(string.printable)
num_layers = 4
learning_rate = 0.01
num_epochs = 20

# 훈련 데이터 정의
input_str = 'hellowolrd'
output_str = 'hellowolrd'

# input_str과 output_str을 리스트로 변환
input_list = list(input_str)

# 리스트를 랜덤하게 섞음
random.shuffle(input_list)

# 섞인 리스트를 다시 문자열로 변환
shuffled_input_str = ''.join(input_list)

print('입력데이터 : ', shuffled_input_str)
print('출력데이터 : ', output_str)

all_chars = string.printable

# 문자를 인덱스로 변환
char2index = dict(zip(all_chars, range(len(all_chars))))
# 인덱스를 문자로 변환
index2char = dict(zip(range(len(all_chars)), all_chars)) 

# RNN 모델 정의
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) 
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = out.reshape(out.size(0)*out.size(1), self.hidden_size)
        out = self.fc(out)
        return out, hidden

# 모델 인스턴스 생성
model = RNN(input_size, hidden_size, output_size, num_layers)

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
# model.parameters() : 모델의 파라미터, lr : 러닝레이트, betas=(0.9, 0.999) : 평균제곱과 제곱근, eps : 숫자 안정성을 위한 값, weight_decay : 가중치 감소
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)

# 훈련 루프
for epoch in range(num_epochs):
    hidden = None
    
    input_tensor = torch.tensor([[char2index[c] for c in input_str]])
    output_tensor = torch.tensor([char2index[c] for c in output_str])
    
    input_one_hot = nn.functional.one_hot(input_tensor, num_classes=len(all_chars)).float()
    
    optimizer.zero_grad()
    output, hidden = model(input_one_hot, hidden)
    loss = criterion(output, output_tensor)
    loss.backward()
    
    # Adam+ 적용
    # 현재 파라미터 값을 이전 값에서 lr * 0.01만큼 감소 시켜 파라미터 업데이트
    for p in model.parameters():
        p.data.mul_(1.0 - learning_rate * 0.01)
    
    optimizer.step()
    
    output_indices = output.argmax(dim=1)
    predicted_chars = ''.join([index2char[i.item()] for i in output_indices])
    accuracy = torch.mean((output_indices == output_tensor).float())
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%, Predicted String: {predicted_chars}')


입력데이터 :  lhodelowrl
출력데이터 :  hellowolrd
Epoch [1/20], Loss: 4.5871, Accuracy: 0.00%, Predicted String: !P!Z44ZPZ4
Epoch [2/20], Loss: 3.5298, Accuracy: 40.00%, Predicted String: hlllllllll
Epoch [3/20], Loss: 2.1342, Accuracy: 30.00%, Predicted String: llllllllll
Epoch [4/20], Loss: 1.7328, Accuracy: 30.00%, Predicted String: llllllllll
Epoch [5/20], Loss: 1.4476, Accuracy: 60.00%, Predicted String: holloooloo
Epoch [6/20], Loss: 1.1402, Accuracy: 50.00%, Predicted String: hellllllll
Epoch [7/20], Loss: 0.8413, Accuracy: 80.00%, Predicted String: hellololod
Epoch [8/20], Loss: 7.0093, Accuracy: 30.00%, Predicted String: hepFheoqFp
Epoch [9/20], Loss: 3.2131, Accuracy: 60.00%, Predicted String: hellooo#qT
Epoch [10/20], Loss: 1.1847, Accuracy: 70.00%, Predicted String: helllwoloo
Epoch [11/20], Loss: 0.9731, Accuracy: 70.00%, Predicted String: helllwoloo
Epoch [12/20], Loss: 0.7727, Accuracy: 80.00%, Predicted String: hellowoloo
Epoch [13/20], Loss: 0.6044, Accuracy: 80.00%, Predicted S